# Setting up Twitter Streaming API

I had to register a Twitter Developers account, which I did as a student, it gave me access right away. After that I needed to set up the stream.

In [1]:
import socket
import sys
import requests
from requests_oauthlib import OAuth1
import json

In [2]:
API_KEY = 'mChKtMIlxucUhULo2VsiXXuz1'
API_SECRET = '8wSiOpCdfY7PAq8JDVUHWjB1z6ltxIJ4rGynvBxVEtfD7DA23L'
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAHf4LQEAAAAAfTYRa4qkKXl1yHSXzSl%2BL0hfAMU%3DjbbxiTPUuSONxcM6b0I2EOTrhNAVlbi4nTwArftzbxCUzyFe4P'

In [3]:
request_token_url = 'https://api.twitter.com/oauth/request_token'

# Using OAuth1 auth helper
oauth = OAuth1(API_KEY, client_secret=API_SECRET)
r = requests.post(url=request_token_url, auth=oauth)
from urllib.parse import parse_qs
credentials = parse_qs(r.content)
resource_owner_key = credentials[b'oauth_token'][0].decode("utf-8") 
resource_owner_secret = credentials[b'oauth_token_secret'][0].decode("utf-8") 

In [4]:
base_authorization_url = 'https://api.twitter.com/oauth/authorize'

# Using OAuth1 auth helper
authorize_url = base_authorization_url + '?oauth_token='
authorize_url = authorize_url + resource_owner_key
print('Please go here and authorize,', authorize_url)

Please go here and authorize, https://api.twitter.com/oauth/authorize?oauth_token=EGs-4AAAAAABLfh3AAABdwIgx4I


In [5]:
verifier = '7799384'

In [6]:
access_token_url = 'https://api.twitter.com/oauth/access_token'

# Using OAuth1 auth helper
oauth = OAuth1(API_KEY,
                   client_secret=API_SECRET,
                   resource_owner_key=resource_owner_key,
                   resource_owner_secret=resource_owner_secret,
                   verifier=verifier)
r = requests.post(url=access_token_url, auth=oauth)
credentials = parse_qs(r.content)
resource_owner_key = credentials.get(b'oauth_token')[0].decode("utf-8")
resource_owner_secret = credentials.get(b'oauth_token_secret')[0].decode("utf-8")

In [7]:
protected_url = 'https://api.twitter.com/1/account/settings.json'

# Using OAuth1 auth helper
oauth = OAuth1(API_KEY,
                   client_secret=API_SECRET,
                   resource_owner_key=resource_owner_key,
                   resource_owner_secret=resource_owner_secret)
r = requests.get(url=protected_url, auth=oauth)

After setting up the API, I created a function, where I could check if it works properly:

In [8]:
def get_tweets():
    url = 'https://stream.twitter.com/1.1/statuses/filter.json'
    query_data = [('language', 'en'), ('locations', '-130,-20,100,50'),('track','#')]
    query_url = url + '?' + '&'.join([str(t[0]) + '=' + str(t[1]) for t in query_data])
    response = requests.get(query_url, auth=oauth, stream=True)
    print(query_url, response)
    return response

In [19]:
resp = get_tweets()

i = 0
for line in resp.iter_lines():
    full_tweet = json.loads(line)
    tweet_text = full_tweet['text']
    print("Tweet Text: " + repr(tweet_text))
    print ("------------------------------------------")
    i += 1
    if i == 5:
        break

https://stream.twitter.com/1.1/statuses/filter.json?language=en&locations=-130,-20,100,50&track=# <Response [200]>
Tweet Text: '“Responders are taught, not born, so make sure their instruction precedes the use of self-assessment and peer-asses… https://t.co/hoUv2iQ8A4'
------------------------------------------
Tweet Text: "@Ness_Graphics New to nft and getting back into visual arts again, here's my first NFT piece\n https://t.co/9MX8l4YjjZ"
------------------------------------------
Tweet Text: '*Nobody tweet me about the fact McGirt is -2 after 3. We can’t jinx this 👀👀👀'
------------------------------------------
Tweet Text: '@reuxzilla Jesus Crice. Lol'
------------------------------------------
Tweet Text: '@stationerscomms @marketors @StephenPlatten @CMAgovUK @stationers @BristowsLawFirm @AMummyToo @nanocelebrity… https://t.co/9Dg4vvGXBE'
------------------------------------------


It is working as intended. Now I have to channel the data into Spark.

In [20]:
def send_tweets_to_spark(http_resp, tcp_connection):
    for line in http_resp.iter_lines():
        try:
            full_tweet = json.loads(line)
            tweet_text = full_tweet['text'] + '\n'
            print("Tweet Text: " + repr(tweet_text))
            print ("------------------------------------------")
            tcp_connection.send((tweet_text).encode('utf-8'))
        except:
            e = sys.exc_info()[0]
            print("Error: %s" % e)

In [ ]:
TCP_IP = "localhost"
TCP_PORT = 37346
conn = None
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((TCP_IP, TCP_PORT))
s.listen(1)
print("Waiting for TCP connection...")
conn, addr = s.accept()
print("Connected... Starting getting tweets.")
resp = get_tweets()
send_tweets_to_spark(resp, conn)

Waiting for TCP connection...
Connected... Starting getting tweets.
https://stream.twitter.com/1.1/statuses/filter.json?language=en&locations=-130,-20,100,50&track=# <Response [200]>
Tweet Text: '@ifamouscici Say less 📍\n'
------------------------------------------
Tweet Text: 'taco bell bring back quesarritos challenge\n'
------------------------------------------
Tweet Text: 'I made $260 off $12 what am I ???\n'
------------------------------------------
Tweet Text: 'I sing this about 30 times a day\n'
------------------------------------------
Tweet Text: "@imjustuhgrl Yes sis shit had me in a dark place cause I can't speak well therefore I can't record but this time ha… https://t.co/rJ4mlJ78Ut\n"
------------------------------------------
Tweet Text: '@ioanahobai Haha. Well, it is abstract enough. This is a nice thought, though!\n'
------------------------------------------
Tweet Text: '@tasialanise And is!\n'
------------------------------------------
Tweet Text: 'I, too, have que

Tweet Text: '@nahashon87 Raila cows just need something no matter how stupid to hang on to. This gives them hope their idol, Rai… https://t.co/8WV4P0lcrO\n'
------------------------------------------
Tweet Text: '@jsrailton @acoyne @bellingcat @joelthesakic @LeiaRx Just ask insiders. Lawmakers and bureaucrats who are Trumpets… https://t.co/b0llfM90qf\n'
------------------------------------------
Tweet Text: '@LeslieMac @atiralyons Well that and you’re gorgeous.\n'
------------------------------------------
Tweet Text: 'Beautiful.\n'
------------------------------------------
Tweet Text: 'Not only a generally great podcast but they have a wonderful guest!\n'
------------------------------------------
Tweet Text: 'The fact that #GalaxyS21 Pro can shoot 4k 60 on all three lenses is crazy. Love seeing the commitment to smart came… https://t.co/Kb56HqKmdS\n'
------------------------------------------
Tweet Text: 'We voted for you Joe, if you pardon President Pendejo Trump, the people will i

Tweet Text: 'Why do gay boys hold grudge in relationships longer than females a woman will be hurt but will heal a gay will drag… https://t.co/KCHJSTxj2O\n'
------------------------------------------
Tweet Text: 'this is what i did last year when the protesters showed up March outside the senate chambers, it was semi-effective.\n'
------------------------------------------
Tweet Text: '@King_of_Gotham Google ay\n'
------------------------------------------
Tweet Text: 'I really like this\n'
------------------------------------------
Tweet Text: '😂 😂 😂 😂 😂 \n\n#Repost @jennaluvs @download.ins\n---\nHe said you gonna get there quick in them big ass shoes💀😭💀😭💀 @ Atl… https://t.co/jF5ne6g1BE\n'
------------------------------------------
Tweet Text: '@beardsley_adam @AkivaMCohen @YonahLieberman @JoeBiden @J_Insider What part of the definition shuts down criticism of Israel?\n'
------------------------------------------
Tweet Text: '🎁🧘🏾\u200d♀️🌌 𝐅𝐈𝐑𝐒𝐓 𝐆𝐈𝐕𝐄𝐀𝐖𝐀𝐘 𝐎𝐅 𝐓𝐇𝐄 𝐘𝐄𝐀𝐑! 🌌🎁🧘🏾\u200d♀️ @wand

Tweet Text: 'Zero sticks a day put life on replay\n'
------------------------------------------
Tweet Text: 'WHOOP HIS ASS SCRAPPY DADDY\n'
------------------------------------------
Tweet Text: 'Ready for a new year? Let’s start 2021 strong with these 5 steps from @YouVersion. https://t.co/ZvQIN3xmJM\n'
------------------------------------------
Tweet Text: '@Chef_Warzu Could definitely get behind this. Sometimes change is good for both sides. Not sure its time for Tomlin… https://t.co/yfG0gCYoZI\n'
------------------------------------------
Tweet Text: '@mmejendouglas Thanks for sharing!!  These decisions can be complicated!!\n'
------------------------------------------
Tweet Text: 'I’ve been on the phone with my boss for almost 3 hours.....my caseload is not even that high for us to be talking t… https://t.co/FNPc1k56qH\n'
------------------------------------------
Tweet Text: '@lee_glidewell On the same subject https://t.co/heNK15zo5l\n'
------------------------------------------


Tweet Text: 'I need a Memphis homie like this\n'
------------------------------------------
Tweet Text: 'also there’s a deaf woman in our class and her and her interpreter are so cute 🥺\n'
------------------------------------------
Tweet Text: '@GlauSciathan @AliceFromQueens @fivealivebysh5 a ‘betrayal’ is strong for this. this is still a lot of money thats its super rare to get.\n'
------------------------------------------
Tweet Text: '@KaroEdor1 @channelstv Is another name for the new virus.\n'
------------------------------------------
Tweet Text: 'Every child deserves a safe space!\n'
------------------------------------------
Tweet Text: 'I ate a significant amount of ABP at the Kendall Square one while I was working at MIT\n'
------------------------------------------
Tweet Text: '@selenagomez My 14 month old is obsessed w/ your music. 🎶💗 https://t.co/82BkT0fw8T\n'
------------------------------------------
Tweet Text: '@lizzardo7 @Acosta You are WAY too kind; I think Fred was a

...

Tweet Text: '@FullTimeHoops1 Spot on\n'
------------------------------------------
Tweet Text: "@blackwomenviews @QueenMab87 They're just the left side of the horseshoe, they've always been closest to the the Ra… https://t.co/luIgs4bIO1\n"
------------------------------------------
Tweet Text: 'Normal convention years we can only allocate so many people to attend due to cost of travel, hotels &amp; food...not th… https://t.co/V5eIYmaA7Y\n'
------------------------------------------
Tweet Text: '@TimBuckleyWX Maybe 2021 Winter can be COLD for NC beginning the 31st! https://t.co/k0wpmGrVZf\n'
------------------------------------------
Tweet Text: '@SlimChonson As you can see even I was viewing from my phone.\n'
------------------------------------------
Tweet Text: "@justussila6 @JunetMohamed @EtalePhilip @DennisOnyangoGM Raila loves flattery no matter how fake, even when it's ob… https://t.co/SnunzpRQ6s\n"
------------------------------------------
Tweet Text: 'Just posted a photo @ Mon

Tweet Text: '@BrandonDavisBD still doesn’t feel real...\n'
------------------------------------------
Tweet Text: 'Join the Dollar Tree team! See our latest job opening here: https://t.co/m5W38QgLCd #Labor #Oakland, CA\n'
------------------------------------------
Tweet Text: '@glitchfur @Vanillayote Kat! Need more bowling stickers x.x hehe\n'
------------------------------------------
Tweet Text: 'Interested in a job in Melbourne, FL? This could be a great fit. Click the link in our bio to apply: Pharmacy Techn… https://t.co/TDG0auFvgI\n'
------------------------------------------
Tweet Text: 'Am I gonna end up at DHS today? \n\nYup.\n'
------------------------------------------
Tweet Text: '@empressrey @zuzusolo @ObiwanxKannoli It says 2020 in the ad, so I doubt something new is coming out.\n'
------------------------------------------
Tweet Text: '@oxox_juju I hope she knows though\n'
------------------------------------------
Tweet Text: 'Yasss ! I love it lol\n'
------------------

Tweet Text: '@BillyBaldwin @Rosie This list omits other unquantifiable but real effects, such as an entire generation of childre… https://t.co/host87dKZi\n'
------------------------------------------
Tweet Text: '@TeamSasha Thank you Team🤗 have a good evening 🥰\n'
------------------------------------------
Tweet Text: 'I’ll do it one day, just not yet.\n'
------------------------------------------
Tweet Text: 'Love You  @sidharth_shukla\n❤️❤️❤️❤️❤️❤️❤️\n\nMaking  chain : 110\nSidharth Shukla \n#SidharthShukla\n'
------------------------------------------
Tweet Text: 'Wow... not the dogs just broke in my room lmao wtf\n'
------------------------------------------
Tweet Text: '@Bethink_RBX Yeyyyyy New Home in Adopt meeeee https://t.co/LLKac3Iazq\n'
------------------------------------------
Tweet Text: 'Just here Celebrating the fine work of my good friend and colleague Neil Armstrong https://t.co/1rna2XEJtd\n'
------------------------------------------
Tweet Text: 'At City Hall, where M

Tweet Text: '@MadieRulla I love you too! I miss you already 🥺 5 months...\n'
------------------------------------------
Tweet Text: 'Who are you to tell me I don’t want you the way flesh wants freedom? The way greed love need. The way kings need kingdoms.\n'
------------------------------------------
Tweet Text: '@conarck Did the $100,000 settlement for police misconduct at the BLM protest come out of JSO budget?\n\nHow many bla… https://t.co/SnT6whxYC5\n'
------------------------------------------
Tweet Text: '@M_everett32 They won’t\n'
------------------------------------------
Tweet Text: 'It would have helped when Pete Buttigieg and Loblaws fixed our bread prices 🍞📈\n'
------------------------------------------
Tweet Text: '@Russ_Agdern i often do a mr. burns impression and wonder whether it just makes me look old\n'
------------------------------------------
Tweet Text: '@NOUNEEE_ Strips and fries thanks\n'
------------------------------------------
Tweet Text: '@NoahCRothman I’ve

Tweet Text: 'Try 6 cousins 🤩\n'
------------------------------------------
Tweet Text: 'My monster had been an awesome bike and it taught me so much. I remember how I got in the highway for the first tim… https://t.co/iPfeyQ72dg\n'
------------------------------------------
Tweet Text: '@curvy_mermaid Share yours!\n'
------------------------------------------
Tweet Text: '@CpaUmuro @LeWano So cool.great idea\n'
------------------------------------------
Tweet Text: 'Years later I’m realizing how depressing this song is.\n'
------------------------------------------
Tweet Text: 'Chick-fil-A ranch nasty asf\n'
------------------------------------------
Tweet Text: '@hroonlikemaroon @YorkdaleMazda @mazdacanada Thanks!!\n'
------------------------------------------
Tweet Text: '@scottscott1e Sorry justice takes time and patience is a virtue.  There are a lot to arrest.\n'
------------------------------------------
Tweet Text: 'Day 1 of my bathroom renovation! 😝😝\n'
------------------------

In [34]:
s.close()